# Test of OpenCV + FastAI for video capture

In [1]:
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *
import fastai

import numpy as np
import cv2 as cv

<br>
<br>

## Load the model

In [2]:
path = Path('../models')
learn_inf = load_learner(path/'200921-0435-FALL-lrfoc4-ulrfoc6')
# learn_inf = load_learner(path/'200921-0600-FALL-lrfoc4-ulrfoc6-F-lrfoc4-ulrfoc6')

<br>
<br>

## Function definitions

In [14]:
def setup_camera() -> cv.VideoCapture:
    '''
    Set up the camera source
    '''
    cap = cv.VideoCapture(0)
    # cap = cv.VideoCapture('http://127.0.0.1:4747/video')
    # cap = cv.VideoCapture('http://10.0.0.75:4747/video')
    # cap = cv.VideoCapture('Droidcam')
    # cap = cv.VideoCapture('http://10.0.0.145:4747/video')
    
    print(f'Autofocus status: {cap.set(cv.CAP_PROP_AUTOFOCUS, 0)}')
    print(f'Manual focus to shortest distance: {cap.set(cv.CAP_PROP_FOCUS, 0)}')

    # Check if camera opened successfully
    assert (cap.isOpened), "Error opening video stream or file!"
    
    return cap




def close_camera():
    cap.release()
    cv.destroyAllWindows()
    
    


def print_stats():
    '''
    Print some basic stats to stdout
    '''
    
    print(f'Frame shape = {frame.shape}')
    print(f'Total number of Frames = {nframe}')
    print(f'Number of frames processed = {n_proc_frames // PROC_NTH_FRAME}')
    
    
    
    
def save_images(frame, prframe):
    '''
    write small and large images to jpeg
    '''
    subdir = 'A'
    prefix = 'oldwebcam'
    
    cv.imwrite(f'../data/frank/{subdir}/{prefix}-S-{n_proc_frames}.jpg', prframe)
    cv.imwrite(f'../data/frank/{subdir}/{prefix}-L-{n_proc_frames}.jpg', frame)
    
    frame = cv.putText(frame, f'Saved Frame #{n_proc_frames}', 
                   org=(20,40), fontFace=cv.FONT_HERSHEY_PLAIN, 
                   fontScale=2, color=(0,255,255), thickness=2,
                   lineType=cv.LINE_AA) 
    
    # Display the frame with the writing on it
    cv.imshow('What the Camera Sees:',frame)
    
    
    
    


<br>
<br>

## Open the video camera, loop for every Frame, massage the image and make a prediction.  

#### This is the main code cel.

In [ ]:
PROC_NTH_FRAME = 1  # Skip every N-1 frames
CAPTURE_IMAGES_TO_FILE = False  # flag if we want to capture the image to disk

cap = setup_camera()

n_proc_frames, nframe = 0,0  # number of frames

# declare here to widen scope
pred, pred_idx, probs = " ", 0, [0.0,0.0,0.0]


# "Game" Loop ... for every frame
while(cap.isOpened()):
        
    nframe += 1
    ret, frame = cap.read()  # Capture frame
    wait_ret = cv.waitKey(5)  # key char value if any

    # Break out of the loop if the user presses 'Q'
    if wait_ret & 0xFF == ord('q'):
        print_stats()
        break
    
    # only process the rest if the capture was successful
    if not ret:
        print("Can't read frame from capture source!")
        break  # break out of loop to let the camera close

    # crop the original 640x480 image down to a centered square 480x480
    frame = frame[:, 80:-80]

    # mirror the image horizontally
    frame = cv.flip(frame, 1)


    # process only if the space key is hit and
    # only process every PROC_NTH_FRAME frames 
    if wait_ret & 0xFF == ord(' ') and (nframe % PROC_NTH_FRAME == 0):
        n_proc_frames +=1  # num processed frames

        # resize the image to 128x128 after "zooming in"
        prframe = cv.resize(frame[60:-60, 60:-60], (128,128))

        # Display the frame that we pass through the predictor
        cv.imshow('What the Predictor Sees:', prframe)

        if CAPTURE_IMAGES_TO_FILE :
            save_images(frame, prframe)
            continue  # no need to predict

        # create the prediction
        pred,pred_idx,probs = learn_inf.predict(prframe)


    # write the prediction as text on the frame
    if probs[pred_idx] > 0.75:
        frame = cv.putText(frame, f'Predict: {pred}, prob: {probs[pred_idx]:.02f}', 
                           org=(20,40), fontFace=cv.FONT_HERSHEY_PLAIN, 
                           fontScale=2, color=(0,255,255), thickness=2,
                           lineType=cv.LINE_AA) 

    # Display the original frame
    cv.imshow('What the Camera Sees:',frame)

# Release the capture object
close_camera()


Autofocus status: False
Manual focus to shortest distance: False


In [15]:
cap.release()
cv.destroyAllWindows()                     

<br>
<br>
<br>
<br>
<br>


# The rest of this file is for reference only.  

<br>
<br>
<br>

#### The code here is more complete than my prototype and I'll want to add it back later.

In [ ]:
import cv
import numpy as np

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv.VideoCapture('Chaplin.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Display the resulting frame
    cv.imshow('Frame',frame)

    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv.destroyAllWindows()



<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


# Snippets

```python
# rotate the image
frame = cv.rotate(frame, cv.ROTATE_90_COUNTERCLOCKWISE)
```